# Working with Boolean data

In [1]:
import numpy as np
import scipy.stats as st
import polars as pl
import os

import cmdstanpy
import arviz as az

import iqplot
import bebi103

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

Loading BokehJS ...

In this problem, we will work with data of the True/False type. Lots of data sets in the biological sciences are like this. For example, we might look at a certain mutation in *Drosophila* that affects development and we might check whether or not eggs hatch.

The data we will use comes from an experiment we have done in past years in [Bi 1x](http://bi1x.caltech.edu/) at Caltech. We studied a neural circuit in *C. elegans* using optogenetics.

A neural circuit is a series of interconnected neurons that create a pathway to transmit a signal from where it is received to where it causes a behavioral response in an animal.  An example is the neural circuit involved in reversals in *C. elegans*.  This circuit consists of three types of neurons: sensory neurons receive stimuli from the environment, command interneurons integrate information from many sensory neurons and pass a signal to the motor neurons, and motor neurons that control worm behavior, such as reversals.

There are six non-motor neurons acting in a circuit that responds to environmental cues and triggers a reversal, a shown in the figure below (based on [Schultheis et. al. 2011](https://doi.org/10.1371/journal.pone.0018766)). These include four sensory neurons (ALM, AVM, ASH, and PLM) and two interneurons (AVD and AVA). Each sensory neuron is sensitive to a different type of stimulus.  For example, the sensory neuron we are studying (ASH) is sensitive to chemosensory stimuli such as toxins, while another neuron (PLM) is sensitive to mechanical stimuli (touch) in the posterior part of the worm's body. The sensory neurons send signals that are integrated by the two command interneurons (AVA and AVD).  Each sensory neuron can provide an impulse to the command interneurons at any time. In order for the command interneuron to fire and activate motor neurons, the sum of the stimuli at any point in time must exceed a certain threshold. Once the stimuli from one or more sensory neurons has induced an action potential in a command interneuron, that signal is passed to motor neurons which will modulate worm behavior.

![Reversal neural network](https://github.com/wis-stats/wis_stats_notebooks_2025/blob/main/ ?raw=1) {#fig-reversal-neural-network width=70%}

In the experiment, we used optogenetics to dissect the function of individual neurons in this circuit.  We worked with two optogenetic worm strains. The ASH strain has Channelrhodopsin (ChR2, represented by a red barrel in the figure above) expressed only in the ASH sensory neuron. When we shine blue light on this strain, we should activate the ChR2, which will allow sodium and calcium cations to flow into the neuron, simulating an action potential. We want to quantify how robustly this stimulation will cause the worm to exhibit aversion behavior and reverse.  

We also studied an AVA strain that has channelrhodopsin expressed only in the AVA command interneuron. Our goal is to quantify the effects of stimulating this neuron in terms of reversals compared to the ASH neuron and to wild type.

The True/False data here are the whether or not the worms undergo a reversal. Here is what the students observed.

|Strain|Year|Trials|Reversals|
|:---:|:---:|:---:|:---:|
|WT|2017|55|7|
|ASH|2017|54|18|
|AVA|2017|52|28|
|WT|2016|36|6|
|ASH|2016|35|12|
|AVA|2016|36|30|
|WT|2015|35|0|
|ASH|2015|35|9|
|AVA|2015|36|33|

For the purposes of this problem, assume that we can pool the results from the three years to have 13/126 reversals for wild type, 39/124 reversals for ASH, and 91/124 reversals for AVA.

Our goal is to estimate $\theta$, the probability of reversal upon exposure to blue light for each strain. That is to say, we want to compute $g(\theta\mid n, N)$, where $n$ is the number of reversals in $N$ trials.

**a)** Propose a reasonable model and then draw samples of $\theta$ out of the posterior distribution using Stan.

**b)** The posterior plots of $\theta$ are illuminating, but suppose we want to quantify *the difference* in reversal probability between the two strains, say strain 1 and strain 2. That is, we want to compute $g(\delta_{12}\mid n_1, N_1, n_2, N_2)$, where $\delta_{12} \equiv \theta_2 - \theta_1$. Plot samples of $\delta_\mathrm{WT,ASH}$, $\delta_\mathrm{WT,AVA}$, and $\delta_\mathrm{ASH,AVA}$.

_____________
# Answer
## a
The story fits N Bernoulli trials with n successes - hence this is a **Binomial distribution**. This distribution has only the parameter $\theta$ which is the probability of success in each trial. Hence, $\theta$ is bounded between 0 and 1 - therefore I chose to sample it out of a **beta** distribution with parameters $\alpha=2\text{, }\beta=2$. Stan file is as follows:
```stan
data {
    int<lower=1> N;
    int<lower=0> n;
}

parameters {
    // probability of sucess
    real<lower=0, upper=1> theta;
}

model {
    // Priors
    theta ~ beta(2, 2);

    // Likelihood
    n ~ binomial(N, theta);
}
```

In [ ]:
stan_code="""
data {
    int<lower=1> N;
    int<lower=0> n;
}

parameters {
    // probability of sucess
    real<lower=0, upper=1> theta;
}

model {
    // Priors
    theta ~ beta(2, 2);

    // Likelihood
    n ~ binomial(N, theta);
}
"""
stan_file_name = 'TomerAntman-77-bluelight_binomial.stan'
if not os.path.exists(stan_file_name):
    with open(stan_file_name, 'w') as f:
        f.write(stan_code)

data = {
    'WT':{ 
    'n': 13, 'N': 126 
    },
    'ASH':{
    'n': 39, 'N': 124
    },
    'AVA':{
    'n': 91, 'N': 124
    }
}

sm = cmdstanpy.CmdStanModel(stan_file=stan_file_name)

15:07:13 - cmdstanpy - INFO - compiling stan file /tmp/tmp_t1hx2sd/tmp31n0zmk9.stan to exe file /home/tomerantman/projects/wis-stats/Statistical inference with Markov chain Monte Carlo/TomerAntman-77-bluelight_binomial
15:07:28 - cmdstanpy - INFO - compiled model executable: /home/tomerantman/projects/wis-stats/Statistical inference with Markov chain Monte Carlo/TomerAntman-77-bluelight_binomial


In [ ]:
# Run sampling for each strain
results = []
for k in data.keys():
    samples = sm.sample(
        data=data[k],
        chains=4,
        iter_sampling=1000,
    )
    samples = az.from_cmdstanpy(posterior=samples)

    thetas = samples.posterior.theta.values.flatten().tolist()
    chains = [str(i) for i in range(4) for _ in range(1000)]

    results.append({
        "strain": k,
        "theta": thetas,
        "chain": chains,
    })

# Combine results into a single DataFrame and explode lists into rows
df = pl.DataFrame(results)
df = df.explode(["theta", "chain"])
df.head()


15:07:28 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

...

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

15:07:28 - cmdstanpy - INFO - CmdStan done processing.


strain,theta,chain
str,f64,str
"""WT""",0.073976,"""0"""
"""WT""",0.136505,"""0"""
"""WT""",0.0881895,"""0"""
"""WT""",0.132041,"""0"""
"""WT""",0.112622,"""0"""


In [ ]:
# Plot the posterior distributions
p = iqplot.histogram(
    data=df,
    q='theta',
    cats='strain',
    arrangement="overlay",
    conf_int=True,
    rug_kwargs={'alpha':0.05},
    frame_width=700,
)
p.legend.title = "Groups"
p.title.text = "Posterior Distributions of Probability of Reversals in Response to Blue-Light"
p.xaxis.axis_label = "Probability of Reversal (θ)"

bokeh.io.show(p)


___________
## b
<!--
**b)** The posterior plots of $\theta$ are illuminating, but suppose we want to quantify *the difference* in reversal probability between the two strains, say strain 1 and strain 2. That is, we want to compute $g(\delta_{12}\mid n_1, N_1, n_2, N_2)$, where $\delta_{12} \equiv \theta_2 - \theta_1$. Plot samples of $\delta_\mathrm{WT,ASH}$, $\delta_\mathrm{WT,AVA}$, and $\delta_\mathrm{ASH,AVA}$.
 -->
 

In [ ]:
# add a column to be able to pivot to wide format
df = df.insert_column(0, pl.Series('draw',[i for i in range(4000)]*3))
df_wide = df.pivot(index='draw', on='strain', values='theta')
# compute the deltas between the samples (there are 4000 samples for each strain and therefore 4000 deltas)
df_wide_gaps = df_wide.select(
    (pl.col('ASH') - pl.col('WT')).alias('WT, ASH'),
    (pl.col('AVA') - pl.col('WT')).alias('WT, AVA'),
    (pl.col('AVA') - pl.col('ASH')).alias('ASH, AVA')
    )
# return to long for easier plotting
df_gap_long = df_wide_gaps.unpivot(variable_name='compared strains', value_name='gap')
# plot histograms of the deltas
p2 = iqplot.histogram(
    data=df_gap_long,
    q='gap',
    cats='compared strains',
    arrangement="overlay",
    conf_int=True,
    rug_kwargs={'alpha':0.05},
    frame_width=700,
)
p2.legend.title = "Comparison of Strains"
p2.title.text = "Differences between the probabilities (θ) of the different strains"
p2.xaxis.axis_label = "Differences in Probability of Reversal [ δ(θ1,θ2) = θ2 - θ1 ]"

bokeh.io.show(p2)


@TomerAntman